In [179]:
#dependencies
import pandas as pd
import numpy as np
import os
from pathlib import Path
import patoolib
import scipy.stats as st

In [28]:
rootdir = Path(r'C:\Users\sakib\Documents\safegraph_data')
#list( rootdir.rglob("*") )

WindowsPath('C:/Users/sakib/Documents/safegraph_data/2020_core/03')

In [4]:
#unzip all of the data and delete archives
'''for child in rootdir.rglob('*'):
    if( child.is_file() and  (child.name[-2:] == 'gz') ):
        patoolib.extract_archive( str( child.absolute() ) , outdir = str(child.parent) )
        child.unlink()
'''

"for child in rootdir.rglob('*'):\n    if( child.is_file() and  (child.name[-2:] == 'gz') ):\n        patoolib.extract_archive( str( child.absolute() ) , outdir = str(child.parent) )\n        child.unlink()\n"

In [ ]:
#Get rid of unnecessary files (just manually named them)
'''
for child in rootdir.rglob('*'):
    if (child.is_file() and (child.name in ('_SUCCESS', 'README.txt', 'brand_info.csv' ) ) ):
        child.unlink()
for child in rootdir.rglob('*'):
    print( child.name )
'''

In [6]:
#puts column in desired position. if out of bounds puts it at beginning/end
#just to help look around at data
def reorder_columns(df, col_name, new_col_idx):
    cols = df.columns.tolist()
    col_idx = cols.index( col_name )
    old_col = cols.pop( col_idx )
    cols.insert(new_col_idx, old_col)
    newdf = df[ cols ]
    return newdf

In [7]:
# filters all relevant sg id's
def filter_bycolumn(df, column, filter_val):
    return df[ df[column] == filter_val  ]['safegraph_place_id']

In [9]:
#same for all Core data?
core_columns = list( ['safegraph_place_id',
 'parent_safegraph_place_id',
 'location_name',
 'safegraph_brand_ids',
 'brands',
 'top_category',
 'sub_category',
 'naics_code',
 'latitude',
 'longitude',
 'street_address',
 'city',
 'region',
 'postal_code',
 'iso_country_code',
 'phone_number',
 'open_hours',
 'category_tags'] )

In [199]:
#Get Core dataframe of a particular month.
#Sped up significantly by selecting a few columns to keep
def core_monthlydata(month):
    rootdir = Path(r'C:\Users\sakib\Documents\safegraph_data')
    month_s = str(month) if(month >= 10) else '0' + str(month)
    core_path = rootdir.joinpath('2020_core', month_s)
    if( not core_path.exists() ):
        raise NotADirectoryError("Input a valid directory/month")
    core_df = pd.DataFrame( columns = core_columns)
    #core_df = pd.DataFrame( columns = ['naics_code', 'safegraph_place_id', 'location_name'] )
    for child in core_path.rglob('*'):
        child_path = child.absolute()
        temp_df = pd.read_csv( child_path )
        #temp_df = pd.read_csv( child_path , usecols = ['naics_code', 'safegraph_place_id', 'location_name'])
        core_df = core_df.append( temp_df , ignore_index = True )
    return core_df

In [200]:
#Load every month into one big dataframe
core_2020 = pd.DataFrame( columns = core_columns)
for i in range(3, 10):
    core_2020 = temp.append( core_monthlydata(i) )
core_2020.shape

MemoryError: Unable to allocate 1.95 GiB for an array with shape (6, 43612331) and data type object

In [139]:
#only loaded these to check for month to month variations
#some months like March for example did not have any data on community food centers. Not sure why?
#df3 = core_monthlydata(3)
#df4 = core_monthlydata(4)
#df5 = core_monthlydata(5)
#df6 = core_monthlydata(6)
#df7 = core_monthlydata(7)
#df8 = core_monthlydata(8)
#df9 = core_monthlydata(9)

In [141]:
#Found 3300 id's for naics_code 624221.
#Matches with the Airtable count for this naics_code.
#core_2020[ core_2020['naics_code'] == 624221 ].count()

naics_code    3300
dtype: int64

In [190]:
# core_2020['naics_code'].isnull().sum() / core_2020['naics_code'].shape[0]
# 0.012428398804260428
# 1.24% of naics_code data (about 470k data points) across Mar-Sep is null. Why is that?

# Causes: unclassified places (a new code), data wasn't collected, data left out on purpose
# which may give insight into the kind of place it is.
# Some of this missing data may be potentially useful. Likely many redundancies

#naics_count = np.zeros( temp['naics_code'].unique().shape[0] )
#for idx, code in enumerate( np.sort( temp['naics_code'].unique() ) ):
#    naics_count[idx] = temp[ temp['naics_code'] == code].count()
#np.mean(naics_count)
# 96493.44186046511
# core_2020['naics_code'].isnull().sum() / np.mean(naics_count)
# 4.870320624271851
# Just taking an average, each naics_code has ~96.5k data points.
# Variance is really high and some codes have over 3 million points so something is wrong here...

# Imputing/predicting these values is probably very difficult as we would need to be very certain to include
# it in our safegraph_id collection. Look at each location?
# Highly unlikely that the missing data is '624221' since we found the 3300 id's.

96493.44186046511

In [121]:
#TODO:
#need to load all data
#for each: reorder columns, filter the two naics_code
#then store all of the safegraph_place_id's (each month has different number of observations)